# Dev Notebook

This notebook loads the version of trulens_eval from the enclosing repo folder. You can use this to debug or devlop trulens_eval features.

In [ ]:
# pip uninstall -y trulens_eval
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens_eval

# trulens_eval notebook dev

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent

print(base)

# If running from github repo, can use this:
sys.path.append(str(base))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

tru.run_dashboard(_dev=base, force=True)

In [ ]:
import lamini
import os
lamini.api_key = os.environ["LAMINI_API_KEY"]

llm = lamini.Lamini("mistralai/Mistral-7B-Instruct-v0.1")
#print(llm.generate("How are you?", output_type={'output': 'string'}))

In [ ]:
from trulens_eval.feedback.prompts import LLM_GROUNDEDNESS_FULL_SYSTEM

In [ ]:
def per_sentence(source, statements):
    outs = []
    for statement in statements.split("."):
        if not statement.strip():
            continue

        print("STATEMENT:", statement)
        comp = llm.generate(LLM_GROUNDEDNESS_FULL_SYSTEM + """
SOURCE: {source}
STATEMENT: {statement}
""".format(source=source, statement=statement), output_type={
    'sentence': 'string',
    'evidence': 'string',
    'score': 'float',
}
        )
        comp['source'] = source
        comp['statement'] = statement
        outs.append(comp)
        print(comp)

    return outs

In [5]:
res = per_sentence(
    source="Poland (Polish: Polska [ˈpɔlska] ⓘ), officially the Republic of Poland,[c] is a country in Central Europe. It is divided into 16 administrative voivodeship provinces, covering an area of 312,696 km2 (120,733 sq mi).[14] Poland has a population of over 38 million and is the fifth most populous member state of the European Union. Warsaw is the nation's capital and largest metropolis. Other major cities include Kraków, Wrocław, Łódź, Poznań, and Gdańsk.",
    statements="Poland is in Europe. Poland has 16 administrative divisions."
)

In [ ]:
res

In [ ]:
from trulens_eval.feedback.provider.lamini import Lamini

In [ ]:
lp = Lamini(model_name="mistralai/Mistral-7B-Instruct-v0.1")

In [ ]:
lp.qs_relevance("Where is Poland?", "Poland is in Europe")

In [ ]:
from trulens_eval.tru_llama import TruLlama

check_keys("OPENAI_API_KEY", "HUGGINGFACE_API_KEY")
import os

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
#os.system(
#    'wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt -P data/'
#)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

# This test does not run correctly if async is used, i.e. not using
# `sync` to convert to sync.

In [ ]:
from trulens_eval.utils.asynchro import sync

tru_query_engine_recorder = TruLlama(query_engine)
llm_response_async, record_async = sync(tru_query_engine_recorder.awith_record,
    query_engine.aquery, "What did the author do growing up?"
)

In [ ]:
record_async

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine)
#with tru_query_engine_recorder as recording:
llm_response_async, record = await tru_query_engine_recorder.awith_record(query_engine.aquery, "What did the author do growing up?")

#record_async = recording.get()

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine)
with tru_query_engine_recorder as recording:
    llm_response_async = query_engine.aquery("What did the author do growing up?")

#record_async = recording.get()

In [ ]:
recording.records

In [ ]:
from llama_index.core.base_query_engine import BaseQueryEngine
isinstance(query_engine, BaseQueryEngine)

In [ ]:
query_engine = index.as_query_engine()
tru_query_engine_recorder = TruLlama(query_engine)
with tru_query_engine_recorder as recording:
    llm_response_sync = query_engine.query(
        "What did the author do growing up?"
    )
record_sync = recording.get()